<a href="https://colab.research.google.com/github/Tatiana164/Univercity-/blob/main/ExtraTask2.2/ExtraTasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Дополнительное задание №2.2. Замыкания. Декораторы. Итераторы. Генераторы**

**БАЗА:**

- **Замыкания** позволяют создавать функции с сохраняющимся состоянием. Это полезно для создания фабричных функций и функций с настраиваемым поведением.
- **Декораторы** позволяют модифицировать или расширять поведение функций без изменения их исходного кода.

---

## **I. Замыкания и декораторы**

### **Пункт №1**

Напишите две функции создания списка из чётных чисел от 0 до N (N – аргумент функции): \([0, 2, 4, ..., N]\).

- **Первая функция** должна использовать метод `append` для добавления элементов в список.
- **Вторая функция** должна использовать **генератор списков** (list comprehensions) для создания списка.

После этого, через **декоратор**, определите время работы этих функций.

In [ ]:
import time
def get_time(fn):
  def inner(*args):
    st_time = time.time()
    res = fn(*args)
    print(f'Время выполнения: {time.time() - st_time:.6f} секунд')
    return res
  return inner
@get_time
def append_fn(N):
  array = []
  for digit in range(0, N, 2):
    array.append(digit)
  return array
@get_time
def gener_fn(N):
  return [value for value in range(0, N, 2)]
print(append_fn(10))
print(gener_fn(10))

---

### **Пункт №2**

Напишите **декоратор** для кэширования результатов работы функции, вычисляющей значение n-го числа [**ряда Фибоначчи**](https://ru.wikipedia.org/wiki/Числа_Фибоначчи).

То есть, при повторном вызове функции через декоратор уже имеющийся результат должен браться из кэша, а не вычисляться заново.

**Например:**

- При значении параметра `n = 5`, должна кэшироваться последовательность \([0, 1, 1, 2, 3, 5]\).
- Вызывая после этого целевую функцию через декоратор ещё раз с `n = 3`, результат \([0, 1, 1, 2]\) должен браться из кэша.
- Если последующее значение `n` больше предыдущего, например `n = 10`, вычисление должно продолжаться, начиная с закэшированной последовательности.

*Подсказка: используйте **замыкание** для хранения кэша внутри декоратора.*


In [ ]:
def fib_cache(fn):
  cache = {}
  def inner(n):
    if n in cache:
      return cache[n]
    result = fn(n)
    for i in range(len(cache), n + 1):
      cache[i] = fn(i)
    return cache[n]
  return inner
@fib_cache
def fib(n):
  if n == 0:
    return 0
  elif n == 1:
    return 1
  else:
    return fib(n - 1) + fib(n - 2)
def fib_seq(n):
  return [fib(i) for i in range(n)]
print(fib_seq(13))
print(f'Кэшировано: {fib(5)}')
print(f'Кэшировано: {fib(3)}')
print(f'Кэшировано: {fib(10)}')

---

### **Пункт №3**

Примените к функции из задания №2 сразу **два декоратора**:

1. **Декоратор**, определяющий время выполнения функции.
2. **Кэширующий декоратор** (из задания №2).

Сравните время работы функции с использованием кэширования и без него.


In [ ]:
import time
def testTime(fn):
  def inner(*args, **kwargs):
    start_time = time.time()
    res = fn(*args, **kwargs)
    end_time = time.time()
    print(f'Время работы: {end_time - start_time:.6f} секунд')
    return res
  return inner
def fib_cache(fn):
  cache = {}
  def inner(n):
    if n in cache:
      return cache[n]
    res = fn(n)
    cache[n] = res
    return res
  return inner
@testTime
def fib_no_cache(n):
  res = []
  a, b = 0, 1
  for _ in range(n):
    res.append(a)
    a, b = b, a + b
  return res
@testTime
@fib_cache
def fib_with_cache(n):
  res = []
  a, b = 0, 1
  for _ in range(n):
    res.append(a)
    a, b = b, a + b
  return res
print('Без кэша:')
print(fib_no_cache(30))
print('\nС кэшем:')
print(fib_with_cache(30))

---

### **Пункт №4**

Создайте функцию **make_multiplier(n)**, которая принимает число **n** и возвращает функцию, умножающую переданное ей число на **n**.

**Пример использования:**

```python
def make_multiplier(n):
    # Ваш код

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15
```

In [ ]:
def make_multiplier(n):
  def inner(value):
    return n * value
  return inner
times3 = make_multiplier(3)
print(times3(5))

---

### **Пункт №5**

Реализуйте функцию с замыканием, которая настраивает округление чисел до заданного количества знаков после запятой.

**Пример:**

```python
def rounder(n):
    # Ваш код

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14
```


In [ ]:
def rounder(n):
  def inner(value):
    return round(value, n)
  return inner
round_to_2 = rounder(2)
print(round_to_2(3.14159))
round_to_3 = rounder(3)
print(round_to_3(3.14159))
round_to_4 = rounder(4)
print(round_to_4(3.14159))

---

### **Пункт №6**

Напишите **декоратор**, который измеряет время исполнения функции и выводит его на экран, только если время превышает определённый порог.

**Пример:**

```python
@time_threshold(threshold=0.5)
def long_running_function():
    # Долгий код

long_running_function()
# Выводится время выполнения только если оно больше 0.5 секунд
```

In [ ]:
import time
def time_threshold(threshold):
  def inner(fn):
    def in_inner(*args):
      st = time.time()
      res = fn(*args)
      itogo_time = time.time()-st
      if itogo_time > threshold:
        print(f'Функция работала: {itogo_time:.6f} секунд.')
      else:
        print(f'Функция работала мало.')
      return res
    return in_inner
  return inner
@time_threshold(0.5)
def long_run_fn(n):
  cnt = 0
  while cnt != n:
    cnt += 1
  return cnt
long_run_fn(78536492)
long_run_fn(1)

---

## **II. Итераторы и генераторы**

---

### **Пункт №1. Генератор строк фиксированной длины**

Напишите генератор `string_generator(char, times)`, который генерирует строки, состоящие из символа `char`, повторенного от 1 до `times` раз.

```python
# Пример использования:
for s in string_generator('*', 5):
    print(s)
# Вывод:
# *
# **
# ***
# ****
# *****
```



---

In [ ]:
def string_generator(value, times):
  string = value
  for _ in range(times):
    yield string
    string += value
for s in string_generator('*', 5):
    print(s)

---

### **Пункт №2. Генератор бесконечной последовательности**

Создайте бесконечный генератор `infinite_sequence()`, который с каждым вызовом возвращает следующее число, начиная с 1.

```python
# Пример использования:
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))
# Вывод:
# 1
# 2
# 3
# 4
# 5
```

---

In [ ]:
def infinite_sequence(start=1):
  while True:
    yield start
    start += 1
gen = infinite_sequence()
for _ in range(5):
  print(next(gen))

---

### **Пункт №3. Генератор комбинированных списков**

Создайте генератор `combined_lists(lst1, lst2)`, который попеременно возвращает элементы из `lst1` и `lst2`. Если длины списков неравны, генератор должен остановиться при исчерпании более короткого списка.

```python
# Пример использования:
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)
# Вывод:
# 1
# 'a'
# 2
# 'b'
# 3
# 'c'
```

---

In [ ]:
def combined_lists(lst1, lst2):
  for a, b in zip(lst1, lst2):
    yield a
    yield b
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
  print(item)

---

### **Пункт №4. Генератор перевернутой строки**

Напишите генератор `reverse_string(s)`, который при каждом вызове возвращает следующий символ строки `s` в обратном порядке.

```python
# Пример использования:
for char in reverse_string('hello'):
    print(char)
# Вывод:
# o
# l
# l
# e
# h
```

---

In [ ]:
def reverse_string(s):
  for char in reversed(s):
    yield char
for char in reverse_string('hello'):
  print(char)

---

### **Пункт №5. Генератор степеней двойки**

Создайте генератор `powers_of_two(n)`, который возвращает степени двойки от 2^0 до 2^n.

```python
# Пример использования:
for num in powers_of_two(5):
    print(num)
# Вывод:
# 1  # 2^0
# 2  # 2^1
# 4  # 2^2
# 8  # 2^3
# 16 # 2^4
# 32 # 2^5
```

---

In [ ]:
def powers_of_two(n):
  for i in range(n + 1):
    yield 2 ** i
for num in powers_of_two(5):
  print(num)

---

### **Пункт №6. Генератор чисел из строки**

Напишите генератор `number_extractor(s)`, который извлекает числа из заданной строки `s` и возвращает их как целые числа.

```python
# Пример использования:
for num in number_extractor('abc123def45gh6'):
    print(num)
# Вывод:
# 123
# 45
# 6
```

---

In [ ]:
def number_extractor(string):
  for sim in string:
        if not sim.isdigit():
          string = string.replace(sim, ' ')
  l = string.split()
  for sim in l:
    yield sim
for num in number_extractor('abc123def45gh6'):
  print(num)

---